In [3]:
### x^5-c*y^5=m

In [7]:
### Using modular arithmetic to check the unsolvability in integers of the equation
### Input: b1,b2,b3, 
###      0<c<b1, 0<abs(m)<b2, and 0<b3 is an indicator of total number of primes considered. Recomend set 2<=b3<=3 first.
### Output: 1 means x^5 - c*y^5 = m is not solvable in integers, 0 means unable to determine or x^5 - c is reducible

def reduction(b1,b2,b3):
    for c in range(1,b1+1):
        for m in range(-b2,b2+1):
            x = polygen(ZZ, 'x')
            poly=x^5 - c
            if poly.is_irreducible() == True:
                div=prime_range(max(c^b3,m^b3))   # larger b3 is, (possibly) more results obtained, but more time needed
                div_c=prime_factors(c)
                for i in range(len(div)):
                    FF=GF(div[i])
                    if div[i] in div_c == True:
                        list=[FF(x^5-m) for x in range(  div[i]  ) ]
                        if prod(list)!=0:
                            return(1)
                        break
                    else:
                        S1=Set([FF(x^5) for x in range(  div[i]  )  ] )
                        S2=Set([FF(c*y^5+m) for y in range(  div[i]  )  ]  )
                        card=S1.intersection(S2).cardinality()
                        if card == 0:
                            return(1)
                        break
                else:
                    return(0)
            else:
                return(0)


In [8]:
### Further sovability check by looking at the norm list
### Input: c,m
### Output: 1 means maybe solvable, 0 means not solvable or the polynomial x^5-c is reducible



def NormCheck(c,m):
    x = polygen(ZZ, 'x')
    poly=x^5 - c
    if poly.is_irreducible() == True:
        K.<theta> = NumberField(poly)
        norm_list=K.elements_of_norm(m)
        if len(norm_list)>0:
            return(1)
        else:
            return(0)
    else:
        return(0)

In [9]:
### Fucntion of modified height, using the approximation with algdep
### Input : X, a list of n non-integer elements; d, the degree of F(x,y)
### Output: the modified heights of each element of X, and the index of [Q(X):Q]

def HeightMod(X,d):
    n=len(X)
    deg=factorial(d)
    P=[CC(X[i]).algdep(deg) for i in range(n)]
    HM=[max( (P[i]).global_height(),  ln(abs(CC(X[i]))),   1   )   for i in range(n)]
    return(HM)

### Outputs the absolute value of elements as considered in complex numbers
### Prvents "bad operand type" error in some cases

def abs_mod(x):
    return(abs(CC(x)))

In [10]:
### Function of the LLL reduction step
### Input: alpha_0, ALPHA=(alpha_1,...,alpha_n),Z0,p_1,p_2
###      as in the text
### Output: the reduced upper bound


def LLLReduction(alpha0,ALPHA,Z0,p1,p2):           
    from sage.functions.log import logb
    C=10^((1.75*logb(Z0, 10).n(prec=40)).ceiling())
    n=len(ALPHA)
    M=matrix.identity(n)
    for i in range(n):
        M[n-2,i]=max( round(C*real_part(  CC(ALPHA[i]).n(prec=50)  )),1 ) 
        M[n-1,i]=max( round(C*imag_part(  CC(ALPHA[i]).n(prec=50)  )),1 )
    Y=zero_vector(n)
    Y[n-2]=round(C*real_part(  CC(alpha0)  ).n(prec=40))
    Y[n-1]=round(C*imag_part(  CC(alpha0)  ).n(prec=40))
    M=M.T
    MLLL=(M.LLL())
    MLLL=MLLL.T
    GS1,GS2=(MLLL).gram_schmidt()
    cc1=max([norm(MLLL.column(0))^2/ norm(GS1.column(i))^2 for i in range(n)])
    sigma=(MLLL).inverse()*Y
    sigma_dist_vec=[min([sigma[i]-floor(sigma[i]),ceil(sigma[i])-sigma[i]]) for i in range(n)]
    sigma_dist_vec=[x for x in sigma_dist_vec if x!= 0]
    sigma_dist=sigma_dist_vec[len(sigma_dist_vec)-1]
    L=cc1^(-1)*sigma_dist*norm(MLLL.column(0))^2
    SS=(n-2)*(Z0)^2
    TT=(1+n*Z0)/sqrt(2)
    upper_bound=floor(abs((1/p2)*(ln(C*p1)-ln(sqrt(L^2-SS)-TT) )))
    return(upper_bound)

In [11]:
### Brute search of integer solution pair (x,y) of x^5-c*y^5=m, up to a given small ran
### Input: ran, a positive integer
### Output: 1 means the equation has such a solution pair, 0 means unable to determine

def search(ran,c,m):
    S1=Set([x^5 for x in range(-ran,ran+1)  ] )
    S2=Set([c*y^5+m for y in range(-ran,ran+1)  ]  )
    card=S1.intersection(S2).cardinality()
    if card > 0:
        return(1)
    else:
        return(0)
                            

In [21]:
### Main function
### Method of Tzanakis and de Weger
### Input: F(x,1), m
###      F(x,1) must be irreducible and monic
### Output: (abs(x),abs(y)) of all integer solutions pairs (x,y)


def solveTzaWeg(F,m):
    PolyRing1.<x> = QQ[]
    poly=F
    deg=poly.degree()
    K.<theta> = NumberField(poly)
    UG = UnitGroup(K)
    TU=[u for u in UG.gens_values() if u.multiplicative_order() < +Infinity]
    FU=[u for u in UG.gens_values() if u.multiplicative_order() == +Infinity]
    rk=len(FU)
    thetas=poly.roots(CC,multiplicities=False)
    norm_list=K.elements_of_norm(m)
    ### Ordering the roots and embeddings so that the real ones are in front, followed by distinct complex ones, then the conjugates 
    
    real_roots=[]
    comp_roots=[]
    for i in thetas:
        if i.is_real() == True:
            real_roots.append(i)
        else:
            comp_roots.append(i)
    r1=len(real_roots)
    r2=(deg-r1)/2
    if r1 < deg:
        ordered_roots=[0 for i in range(deg)]
    for i in range(r1):
        ordered_roots[i]=real_roots[i]
    index=r1
    for i in range(2*r2):
        if comp_roots[i] not in ordered_roots :
            ordered_roots[index]=comp_roots[i]
            ordered_roots[index+r2]=comp_roots[i].conjugate()
            index=index+1
    thetas=ordered_roots
    from sage.rings.number_field.number_field_morphisms import create_embedding_from_approx
    emb=[create_embedding_from_approx(K, thetas[i]) for i in range(deg) ]
    
    
    ### computing c1
    M=matrix(RR,deg,deg)
    for i in range(deg):
        for j in range(deg):
            if j < i:
                M[i,j] = abs(thetas[i]-thetas[j])
            else:
                M[i,j] = abs(thetas[i])+abs(thetas[j])
    c1=2/min(M.list())
    ### Computing c2, c3
    L=[0 for i in range(deg)]
    for i in range(deg):
        L[i]=max([abs((thetas[i]-thetas[j])/(thetas[i]-thetas[k])).n(prec=50) for j in range(deg) for k in range(deg) if j != i if k != j if k != i ])
    c2=max(L)
    c3=c1*c2
    ### Computing c4, c5
    MFU = matrix(RR, rk+1, rk, lambda i, j: ln(abs(emb[i](FU[j]).n(prec=50))))   ### Matrix of log of abs of fundamental units
    V=[(MFU.delete_rows([i])).inverse() for i in range(rk+1)]    ### Inverse of matrix U_I as in the text
    c4=1/(max([V[i].norm(Infinity) for i in range(rk+1)] ))
    c5=c4/(deg-0.9)
    ### Cases A,B 
    ### Computing c6, c7 and the corresponding A1,A2
    M=matrix(RR,deg,len(norm_list))
    for i in range(deg):
        for j in range(len(norm_list)):
            M[i,j]=( abs( emb[i](norm_list[j]) .n(prec=60) ))^(-1)         
    c6=max(M.list())
    c7=min(M.list())
    A1=ln(c6*abs(m))/(c4-(deg-1)*c5)
    A1_min=ln(c6*abs(m))/(c4)
    A2=ln(c7)/(c4-c5)
    ### Case C
    ### Loop all possible abs(theta_i) in the real roots 
    bound_temp=[A1_min,A2,3]
    CC100=ComplexField(100)
    for i in range(deg):
        if thetas[i].is_real() == True:
            JK=[j for j in range(deg) if j != 1]
            j=JK[deg-2]
            k=JK[deg-3]
            alpha2=CC100((emb[i](norm_list[0])/emb[j](norm_list[0])))*CC100((emb[i](FU[0])/emb[j](FU[0])))*((thetas[j]-thetas[k])/(thetas[k]-thetas[i]))
            units=[]
            for ii in range(rk):
                units.append((emb[k](FU[ii])/emb[j](FU[ii])))
            vector=units
            vector.append(alpha2)
            heights=HeightMod( vector ,deg)
            hm0=abs(ln(-1))
            c8=(18*factorial(5)*(4^5)*(256^6)*ln(64)*prod(heights)*hm0)
            A_0=(2/c5)*( ln(2*c3) +c8*ln(3/2) +c8*ln(c8/c5)  ).n(prec=16)
            alpha0=ln(-alpha2)
            ALPHA=units
            ALPHA.append(2*pi*sqrt(-1))
            p1=2*c3
            p2=c5
            bound1=LLLReduction(alpha0,ALPHA,A_0,p1,p2)
            if bound1 >= 50:
                bound2=LLLReduction(alpha0,ALPHA,bound1,p1,p2)
                if bound2 >= 50:
                    bound3=LLLReduction(alpha0,ALPHA,bound2,p1,p2)
                else:
                    bound3=bound2
            else:
                bound3=bound1
            bound_temp.append(bound3)
    bound_final=floor(max(bound_temp))
    
    XY=[]
    from itertools import product
    per=list(product(range(-bound_final,bound_final+1), repeat=rk))   ### Repeated permutation
    for i in range(len(per)):
        result1=prod([FU[j]^(per[i][j]) for j in range(rk)])
        for k in range(len(norm_list)):
            for ii in range(len(TU)):
                result2=(norm_list[k])*(TU[ii])*result1
                P=result2.polynomial(var='t')
                if P.degree() <2:
                    a0=abs(result2[0])
                    a1=abs(result2[1])
                    XY.append((a0,a1))
    return(sorted(set(XY)))

In [13]:
### Preliminary algorithm
### x^5 - c*y^5 = m in Z[x,y], where abs(c)<=n and abs(m)<=n, and WLOG c,m>0

n=10
CCC=[i for i in range(1,n+1)]   # c
MMM=[i for i in range(1,n+1)]   # m
L1=[]
L2=[]
for i in range(len(CCC)):
    x = polygen(ZZ, 'x')
    poly=x^5 - CCC[i]
    if poly.is_irreducible() == True:
        for j in range(len(MMM)):
            if search(20,CCC[i],MMM[j]) == 1:
                L1.append((CCC[i],MMM[j]))
            else:
                const=NormCheck(CCC[i],MMM[j])
                if const==1:
                    if reduction(CCC[i],MMM[j],2) == 0:
                        L2.append((CCC[i],MMM[j]))
                    

print('total number of equations considered =',len(CCC)*len(MMM))
print('out of which are solvable by brute search of solution up to 20=',len(L1))
print('out of which passed the solvability test and hence are up to further inspection =',len(L2))

total number of equations considered = 100
out of which are solvable by brute search of solution up to 20= 34
out of which passed the solvability test and hence are up to further inspection = 56


In [14]:
print(L1)

[(2, 1), (2, 2), (2, 3), (3, 1), (3, 2), (3, 3), (3, 4), (4, 1), (4, 3), (4, 4), (4, 5), (5, 1), (5, 4), (5, 5), (5, 6), (6, 1), (6, 5), (6, 6), (6, 7), (7, 1), (7, 6), (7, 7), (7, 8), (8, 1), (8, 7), (8, 8), (8, 9), (9, 1), (9, 8), (9, 9), (9, 10), (10, 1), (10, 9), (10, 10)]


In [15]:
print(L2)

[(2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 2), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 2), (5, 3), (5, 7), (5, 8), (5, 9), (5, 10), (6, 2), (6, 3), (6, 4), (6, 8), (6, 9), (6, 10), (7, 2), (7, 3), (7, 4), (7, 5), (7, 9), (7, 10), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 10), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (10, 2), (10, 3), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8)]


In [22]:
### Verify no solution exists for the combinations in L2


L2_sol=[]
for i in range(len(L2)):
    F=x^5-L2[i][0]
    sol=solveTzaWeg(F,L2[i][1])
    L2_sol.append((L2[i],sol))
    print((L2[i],sol))

((2, 4), [])


((2, 5), [])


((2, 6), [])


((2, 7), [])


((2, 8), [])


...

((10, 4), [])


((10, 5), [])


((10, 6), [])


((10, 7), [])


((10, 8), [])


In [23]:
### Find all solutions for the combinations in L1
### Took about 80 secs on my laptop


L1_sol=[]
for i in range(len(L1)):
    F=x^5-L1[i][0]
    sol=solveTzaWeg(F,L1[i][1])
    L1_sol.append((L1[i],sol))
    print((L1[i],sol))

((2, 1), [(1, 0), (1, 1)])


((2, 2), [(0, 1)])


((2, 3), [(1, 1)])


((3, 1), [(1, 0)])


((3, 2), [(1, 1)])


...

((9, 9), [(0, 1)])


((9, 10), [(1, 1)])


((10, 1), [(1, 0)])


((10, 9), [(1, 1)])


((10, 10), [(0, 1)])


In [0]:
### Method of Bilu and Hanrot
### Method of Bilu and Hanrot
### Method of Bilu and Hanrot

In [32]:
### The method of Bilu and Hanrot
### Input :F(x,1), m
###      F(x,1) must be irreducible and monic
### Output: absolute values of the solution pairs



def solveBiluHanrot(F,m):
    CC500=ComplexField(500)
    ComplexNumber=CC500
    PolyRing1.<x> = QQ[]
    poly=F
    deg=poly.degree()
    K.<theta> = NumberField(poly)
    UG = UnitGroup(K)
    TU=[u for u in UG.gens_values() if u.multiplicative_order() < +Infinity]
    FU=[u for u in UG.gens_values() if u.multiplicative_order() == +Infinity]
    rk=len(FU)
    thetas=poly.roots(CC,multiplicities=False)
    norm_list=K.elements_of_norm(m)
    ### Ordering the roots and embeddings so that the real ones are in front, followed by distinct complex ones, then the conjugates 
    
    real_roots=[]
    comp_roots=[]
    for i in thetas:
        if i.is_real() == True:
            real_roots.append(i)
        else:
            comp_roots.append(i)
    r1=len(real_roots)
    r2=(deg-r1)/2
    if r1 < deg:
        ordered_roots=[0 for i in range(deg)]
    for i in range(r1):
        ordered_roots[i]=real_roots[i]
    index=r1
    for i in range(2*r2):
        if comp_roots[i] not in ordered_roots :
            ordered_roots[index]=comp_roots[i]
            ordered_roots[index+r2]=comp_roots[i].conjugate()
            index=index+1
    thetas=ordered_roots
    from sage.rings.number_field.number_field_morphisms import create_embedding_from_approx
    emb=[create_embedding_from_approx(K, thetas[i]) for i in range(deg) ]
    
    
    ### computing c1
    M=matrix(RR,deg,deg)
    for i in range(deg):
        for j in range(deg):
            if j < i:
                M[i,j] = abs(thetas[i]-thetas[j])
            else:
                M[i,j] = abs(thetas[i])+abs(thetas[j])
    c1=2/min(M.list())
    ### Computing c2, c3
    ### Might take long time, as there are deg^3 computations
    L=[0 for i in range(deg)]
    for i in range(deg):
        L[i]=max([abs((thetas[i]-thetas[j])/(thetas[i]-thetas[k])) for j in range(deg) for k in range(deg) if j != i if k != j if k != i ])
    c2=max(L)
    c3=c1*c2
    ### Computing c4, c5
    MFU = matrix(RR, rk+1, rk, lambda i, j: ln(abs(emb[i](FU[j]).n(prec=50))))   ### Matrix of log of abs of fundamental units
    V=[(MFU.delete_rows([i])).inverse() for i in range(rk+1)]    ### Inverse of matrix U_I as in the text
    c4=1/(max([V[i].norm(Infinity) for i in range(rk+1)] ))
    c5=c4/(deg-0.9)
    ### Cases A,B 
    ### Computing c6, c7 and the corresponding A1,A2
    M=matrix(RR,deg,len(norm_list))
    for i in range(deg):
        for j in range(len(norm_list)):
            M[i,j]=( abs( CC(emb[i](norm_list[j]))  ))^(-1)         
    c6=max(M.list())
    c7=min(M.list())
    A1=ln(c6*abs(m))/(c4-(deg-1)*c5)
    A1_min=ln(c6*abs(m))/(c4)
    A2=ln(c7)/(c4-c5)
    ### Case C
    ### Computing c8 and an initial bound on A
    
    
    bound_temp=[A1_min,A2,3]
    CC100=ComplexField(100)
    for i in range(deg):
        if thetas[i].is_real() == True:
            JK=[j for j in range(deg) if j != 1]
            j=JK[deg-2]
            k=JK[deg-3]
            alpha2=CC100((emb[i](norm_list[0])/emb[j](norm_list[0])))*CC100((emb[i](FU[0])/emb[j](FU[0])))*((thetas[j]-thetas[k])/(thetas[k]-thetas[i]))
            units=[]
            for ii in range(rk):
                units.append((emb[k](FU[ii])/emb[j](FU[ii])))
            vector=units
            vector.append(alpha2)
            heights=HeightMod( vector ,deg)
            hm0=abs(ln(-1))
            c8=(18*factorial(5)*(4^5)*(256^6)*ln(64)*prod(heights)*hm0)
            A_0=RR((2/c5)*( ln(2*c3) +c8*ln(3/2) +c8*ln(c8/c5)  )).ceil()
    
    
            MFU = matrix( CC500, rk+1, rk, lambda i, j: ln( abs(CC500(emb[i](FU[j]))) ) )  
            II=[]
            for i in range(rk+1):
                II.append(  [i,[j for j in range(rk+1) if j!=i]]   )
            V=[(MFU.delete_rows([i])).inverse() for i in range(rk+1)]    ### V_I, inverse of matrix U_I as in the text
            K.<theta> = NumberField(poly) ### Redefine theta as in the number field, otherwise it's recognised as in the symbolic ring
            import numpy
            delta=[sum(numpy.abs(V[i].row(j).list())) for i in range(rk+1) for j in range(rk)]
            u=norm_list[0]
            LL=[[ln( abs_mod(  ((emb[II[i][0]](theta)- emb[j](theta))/emb[j](u)) )   ) for j in II[i][1] ] for i in range(rk+1)]
            LV=[(MFU.delete_rows([i])).inverse() for i in range(rk+1)]
            for i in range(rk):
                for j in range(rk+1):
                    LV[j].set_col_to_multiple_of_col(i, i, LL[j][i])
            ### Calculating c9, c10, c11, lamdba_k's
            lam=[sum(LV[i].row(j)) for i in range(rk+1) for j in range(rk)]
            c10=max(abs(delta[0]),abs(delta[1]))
            c11=max(((abs(V[0][0,0])+abs(V[0][0,1]))/deg )+abs(lam[0]),  ((abs(V[0][1,0])+abs(V[0][1,1]))/deg )+abs(lam[1]))
            c9=max([2^(deg-1)*abs(m)/abs(poly.derivative()(thetas[i])) for i in range(deg)])
            M=matrix(CC,deg,deg)
            for i in range(deg):
                for j in range(deg):
                    if j < i:
                        M[i,j] = abs(thetas[i]-thetas[j])
                    else:
                        M[i,j] = abs(thetas[i])+abs(thetas[j])
            c1=2/min(M.list())
            c14=c1*c9*e^(deg*c11/c10)
            c15=deg/c10
            c16=c14*(V[0].norm(Infinity))
            ### Calculating S_g, T_g
            delta_h=max(abs(delta[0]),abs(delta[1]))
            h=0
            if delta_h in delta == True:
                h=delta.index(delta_h)
            else:
                if -delta_h in delta == True:
                    h=delta.index(-delta_h)
            if h == 0 :
                g=1
            else:
                g=0
            S_g=delta[g]/delta[h]
            T_g=(delta[g]*lam[h]-delta[h]*lam[g])/delta[h]
            k=50
            ### convergent of S_g with the smallest denominator > X0
            ### Funtion to find q as in the text, using continued fractions
            def cfreduction(S_g,A_0,k):
                X0=k*A_0
                index=0
                c=continued_fraction(S_g)
                while abs(c.numerator(index)-(S_g)*(c.denominator(index))) > 1/X0:
                    index=index+1
                    if c.denominator(index) > X0:
                        break
                q=c.denominator(index)
                return(q,index)
            k=50
            q=cfreduction(S_g,A_0,k)[0]
            index=cfreduction(S_g,A_0,k)[1]
            RR500=RealField(500)
            dist=min(  RR500(q*(T_g)).ceil()-q*(T_g),  q*(T_g)-RR500(q*(T_g)).floor()  )   ### distance to neareast integer of q*T_g
            while dist < 2/k:
                k=2*k
                q=cfreduction(S_g,A_0,k)
                if k > 10000:
                    break
            new_bound=RR((1/c15)*ln(2*c16*k^2*A_0)).ceil() 
            bound_temp.append(new_bound)
    bound_final=floor(max(bound_temp))
    
    XY=[]
    from itertools import product
    per=list(product(range(-bound_final,bound_final+1), repeat=rk))   ### Repeated permutation
    for i in range(len(per)):
        result1=prod([FU[j]^(per[i][j]) for j in range(rk)])
        for k in range(len(norm_list)):
            for ii in range(len(TU)):
                result2=(norm_list[k])*(TU[ii])*result1
                P=result2.polynomial(var='t')
                if P.degree() <2:
                    a0=abs(result2[0])
                    a1=abs(result2[1])
                    XY.append((a0,a1))
    
    ### Computing Y2 and loop over all abs(y)<=Y2
    min_roots_diff=min([abs(thetas[i]-thetas[j]) for i in range(deg) for j in range(deg) if j!=i])
    Y2=RR((((deg+1)*c9)/min_roots_diff)^(1/deg))
    Y2=Y2.floor()
    for jj in range(Y2):
        poly1=F.homogenize(y)
        poly2=PolyRing1(poly1(x,jj))
        int_roots=poly.roots(ZZ,multiplicities=False)
        number1=len(int_roots)
        if number1 > 0:
            for kk in range(number1):
                XY.append((int_roots[kk],jj))
    return(sorted(set(XY)))

In [34]:
### Find all solutions for the combinations in L1, with the method of B&H
### Took about 70 secs on my laptop


L1_sol_BH=[]
for i in range(len(L1)):
    F=x^5-L1[i][0]
    sol=solveBiluHanrot(F,L1[i][1])
    L1_sol_BH.append((L1[i],sol))
    print((L1[i],sol))

((2, 1), [(1, 0), (1, 1)])


((2, 2), [(0, 1)])


((2, 3), [(1, 1)])


((3, 1), [(1, 0)])


((3, 2), [(1, 1)])


...

((9, 9), [(0, 1)])


((9, 10), [(1, 1)])


((10, 1), [(1, 0)])


((10, 9), [(1, 1)])


((10, 10), [(0, 1)])
